<a href="https://colab.research.google.com/github/jacobgreen4477/The-4th-ETRI-AI-Human-Understanding-Competition/blob/main/202_%EC%8B%A4%ED%97%98_v2_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In our study, we used smartphones, smartwatches, sleep sensors, and self-recording apps to collect daily life logs and sleep health records of study participants in 2024.The data collection procedures and methods followed a similar approach to those used in previous studies. Here, we pu﻿blicly provide the following 12 data items, which comprise a total of 700 days' worth of lifelog data, strictly for non-commercial and academic research purposes only.
- mACStatus: Indicates whether the smartphone is currently being charged.
- mActivity: Value calculated by the Google Activity Recognition API.
- mAmbience: Ambient sound identification labels and their respective probabilities.
- mBle: Bluetooth devices around individual subject.
- mGps: Multiple GPS coordinates measured within a single minute using the smartphone.
- mLight: Ambient light measured by the smartphone.
- mScreenStatus: Indicates whether the smartphone screen is in use.
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mWifi: Wifi devices around individual subject.
- wHr: Heart rate readings recorded by the smartwatch.
- wLight: Ambient light measured by the smartwatch.
- wPedo: Step data recorded by the smartwatch.

For the purpose of training a learning model to predict sleep health, fatigue, and stress, the following six metrics were derived from sleep sensor data and self-reported survey records. Each metric consists of values categorized into either two levels (0, 1) or three levels (0, 1, 2), depending on the specific metric. The detailed classification criteria for each metric's levels will be provided in a separate document.These
metrics assign a value of 0 for sleep records that do not meet the recommended guidelines.For instance, the first questionnaire metric (Q1) is assigned a value of 1 on days when an
individual’s self-reported sleep quality exceeds their average over the experimental period, and 0 when it
falls below that average. Similarly, the second and third metrics (Q2 and Q3) are assigned a value of 0
on days when the participant’s fatigue and stress levels, respectively, exceed their average, and a value of
1 when these levels are below average.

- Q1: Overall sleep quality as perceived by a subject immediately after waking up.
- Q2: Physical fatigue of a subject just before sleep.
- Q3: Stress level experienced by a subject just before sleep.
- S1: Adherence to sleep guidelines for total sleep time (TST).
- S2: Adherence to sleep guidelines for sleep efficiency (SE).
- S3: Adherence to sleep guidelines for sleep onset latency (SOL, or SL).

수면 건강, 피로, 스트레스 예측을 위한 학습 모델을 훈련시키기 위해, 수면 센서 데이터와 자기 보고식 설문 기록을 기반으로 다음의 6가지 지표를 도출했습니다.
각 지표는 해당 항목에 따라 두 수준(0, 1) 또는 세 수준(0, 1, 2)으로 구분된 값을 가집니다.
각 지표의 세부 분류 기준은 별도의 문서에서 제공될 예정입니다.

- Q1: 기상 직후 본인이 인지한 전반적인 수면의 질
 - 0: 개인 평균 이하
 - 1: 개인 평균 이상
- Q2: 취침 직전 본인이 느낀 신체적 피로 수준
 - 0: 높은 피로 수준
 - 1: 낮은 피로 수준
- Q3: 취침 직전 본인이 느낀 스트레스 수준
 - 0: 높은 스트레스 수준
 - 1: 낮은 스트레스 수준
- S1: 총 수면 시간(TST) 가이드라인을 준수했는지 3LEVELS
 - 0: 가이드라인 미준수
 - 1: 가이드라인 부분적 준수
 - 2: 가이드라인 완전 준수
- S2: 수면 효율(SE) 가이드라인을 준수했는지 여부
- (SE: 잠자리에 누워 있었던 전체 시간 대비, 실제로 잠든 시간의 비율)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수
- S3: 수면 잠들기 지연 시간(SOL 또는 SL) 가이드라인을 준수했는지 여부
- (SOL: 잠자리에 누운 순간부터 실제로 잠드는 데까지 걸린 시간)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수

### 📦 라이브러리

In [ ]:
! pip install haversine
! pip install optuna
! pip install category_encoders
! pip install timm
import timm
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
from collections import Counter
from scipy.stats import entropy
from haversine import haversine  # 설치 필요: pip install haversine

warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import ast
from tqdm import tqdm  # ← 추가
from math import radians, cos, sin, asin, sqrt
from datetime import time
from datetime import timedelta
from functools import reduce
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import glob
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# seed 고정
SD = 42
random.seed(SD)
np.random.seed(SD)
os.environ['PYTHONHASHSEED'] = str(SD)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [ ]:
def correct_lifelog_date_for_midnight(df, timestamp_col='timestamp', lifelog_col='lifelog_date'):
    df = df.copy()
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df[lifelog_col] = pd.to_datetime(df[lifelog_col])

    # 조건: timestamp의 시(hour)가 0~5시인 경우만 하루 차감
    mask = (df[timestamp_col].dt.hour >= 0) & (df[timestamp_col].dt.hour < 6)
    df.loc[mask, lifelog_col] = df.loc[mask, lifelog_col] - pd.Timedelta(days=1)

    # lifelog_date를 문자열로 바꾸는 경우
    df[lifelog_col] = df[lifelog_col].dt.date.astype(str)

    return df

In [ ]:
import pandas as pd
from io import StringIO

string = """
subject_id	sleep_date
id01	2024-07-24
id01	2024-07-27
id01	2024-08-18
id01	2024-08-19
id01	2024-08-20
id01	2024-08-21
id01	2024-08-22
id01	2024-08-24
id01	2024-08-25
id01	2024-08-26
id01	2024-08-27
id01	2024-08-28
id01	2024-08-29
id01	2024-08-30
id02	2024-08-23
id02	2024-08-24
id02	2024-09-16
id02	2024-09-17
id02	2024-09-19
id02	2024-09-20
id02	2024-09-21
id02	2024-09-22
id02	2024-09-23
id02	2024-09-24
id02	2024-09-25
id02	2024-09-26
id02	2024-09-27
id02	2024-09-28
id03	2024-08-30
id03	2024-09-01
id03	2024-09-02
id03	2024-09-03
id03	2024-09-05
id03	2024-09-06
id03	2024-09-07
id04	2024-09-03
id04	2024-09-04
id04	2024-09-05
id04	2024-09-06
id04	2024-09-07
id04	2024-09-08
id04	2024-09-09
id04	2024-10-08
id04	2024-10-09
id04	2024-10-10
id04	2024-10-11
id04	2024-10-12
id04	2024-10-13
id04	2024-10-14
id05	2024-10-19
id05	2024-10-23
id05	2024-10-24
id05	2024-10-25
id05	2024-10-26
id05	2024-10-27
id05	2024-10-28
id06	2024-07-25
id06	2024-07-26
id06	2024-07-27
id06	2024-07-28
id06	2024-07-29
id06	2024-07-30
id06	2024-07-31
id07	2024-07-07
id07	2024-07-08
id07	2024-07-09
id07	2024-07-10
id07	2024-07-11
id07	2024-07-12
id07	2024-07-13
id07	2024-07-30
id07	2024-08-01
id07	2024-08-02
id07	2024-08-03
id07	2024-08-04
id07	2024-08-05
id07	2024-08-06
id08	2024-08-28
id08	2024-08-29
id08	2024-08-30
id08	2024-08-31
id08	2024-09-01
id08	2024-09-02
id08	2024-09-04
id09	2024-08-02
id09	2024-08-22
id09	2024-08-23
id09	2024-08-24
id09	2024-08-25
id09	2024-08-27
id09	2024-08-28
id09	2024-08-29
id09	2024-08-30
id09	2024-08-31
id09	2024-09-01
id09	2024-09-02
id09	2024-09-03
id09	2024-09-04
id10	2024-08-28
id10	2024-08-30
id10	2024-08-31
id10	2024-09-01
id10	2024-09-02
id10	2024-09-03
id10	2024-09-06
"""

# DataFrame 생성
valid_ids = pd.read_csv(StringIO(string), sep='\t')
valid_ids['pk'] = valid_ids['subject_id']+valid_ids['sleep_date']

### 📦 데이터 읽기

In [ ]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

In [ ]:
%%time

mACStatus['lifelog_date'] = mACStatus['timestamp'].astype(str).str[:10]
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]

# def extract_labels_and_probs(row):
#     items = row['m_ambience']
#     labels = [item[0] for item in items]
#     probs = [item[1] for item in items]
#     return pd.Series({'labels': labels, 'prob': probs})

# mAmbience[['labels', 'prob']]  = mAmbience.apply(extract_labels_and_probs, axis=1)
# mAmbience['lifelog_date'] = mAmbience['timestamp'].astype(str).str[:10]
# mAmbience = mAmbience.drop(columns=['m_ambience'])

# def extract_mble_info(row):
#     m_data = row['m_ble']
#     address = [item['address'] for item in m_data]
#     device_class = [item['device_class'] for item in m_data]
#     rssi = [item['rssi'] for item in m_data]
#     return pd.Series({'address': address, 'device_class': device_class, 'rssi': rssi})

# mBle[['address','device_class','rssi']] = mBle.apply(extract_mble_info, axis=1)
# mBle['lifelog_date'] = mBle['timestamp'].astype(str).str[:10]

# def extract_gps_info(row):
#     m_data = row['m_gps']
#     altitude = [item['altitude'] for item in m_data]
#     latitude = [item['latitude'] for item in m_data]
#     longitude = [item['longitude'] for item in m_data]
#     speed = [item['speed'] for item in m_data]
#     return pd.Series({'altitude': altitude, 'latitude': latitude, 'longitude': longitude, 'speed': speed})

# mGps[['altitude','latitude','longitude','speed']] = mGps.apply(extract_gps_info, axis=1)
# mGps['lifelog_date'] = mGps['timestamp'].astype(str).str[:10]
# mGps = mGps.drop(columns=['m_gps'])

mLight['lifelog_date'] = mLight['timestamp'].astype(str).str[:10]
mScreenStatus['lifelog_date'] = mScreenStatus['timestamp'].astype(str).str[:10]

# def extract_mUsageStats_info(row):
#     m_data = row['m_usage_stats']
#     app_name = [item['app_name'] for item in m_data]
#     total_time = [item['total_time'] for item in m_data]
#     return pd.Series({'app_name': app_name, 'total_time': total_time})

# mUsageStats[['app_name', 'total_time']] = mUsageStats.apply(extract_mUsageStats_info, axis=1)
# mUsageStats['lifelog_date'] = mUsageStats['timestamp'].astype(str).str[:10]

# def extract_wifi_info(row):
#     wifi_data = row['m_wifi']
#     bssids = [item['bssid'] for item in wifi_data]
#     rssis = [item['rssi'] for item in wifi_data]
#     return pd.Series({'bssid': bssids, 'rssi': rssis})

# mWifi[['bssid', 'rssi']] = mWifi.apply(extract_wifi_info, axis=1)
# mWifi['lifelog_date'] = mWifi['timestamp'].astype(str).str[:10]

wHr['lifelog_date'] = wHr['timestamp'].astype(str).str[:10]
wLight['lifelog_date'] = wLight['timestamp'].astype(str).str[:10]
wPedo['lifelog_date'] = wPedo['timestamp'].astype(str).str[:10]

CPU times: user 7.56 s, sys: 906 ms, total: 8.46 s
Wall time: 8.35 s


### 📌 이미지 생성
- spleeptime만 추출 (00시부터 06시까지)
- 참고 : https://github.com/seongjiko/Pixleep/tree/master

In [ ]:
def filter_by_group_size(df, group_cols=['subject_id', 'lifelog_date']):
    # 그룹별 건수 계산
    group_counts = df.groupby(group_cols).size().reset_index(name='count')
    # 평균 건수 계산
    mean_count = group_counts['count'].mean()
    # 평균 초과 그룹만 추출
    valid_groups = group_counts[group_counts['count'] > mean_count*0.5][group_cols]
    # 원본과 inner join으로 필터링
    return df.merge(valid_groups, on=group_cols, how='inner')

def make_timestamps_unique(df, timestamp_col='timestamp'):
    # 'timestamp' 컬럼을 기준으로 정렬
    df = df.sort_values(by=[timestamp_col])
    # 각 'timestamp'가 중복된 횟수를 세어 나노초 단위로 증가시킴
    df[timestamp_col] = df[timestamp_col] + pd.to_timedelta(df.groupby(timestamp_col).cumcount(), unit='ns')
    return df

def average_list_columns(df, list_columns, pk_cols=['subject_id', 'lifelog_date']):

    for col in list_columns:

        def safe_mean(x):
            if isinstance(x, list):
                return np.mean(x) if len(x) > 0 else np.nan
            elif isinstance(x, (int, float, np.integer, np.floating, type(None))):
                return x
            elif isinstance(x, (np.ndarray, pd.Series)):
                return np.mean(x)
            elif pd.api.types.is_scalar(x) and pd.isna(x):
                return np.nan
            else:
                return np.nan

        df[col] = df[col].apply(safe_mean)

    return df

def center_list_values(df, list_columns):
    for col in list_columns:
        def center(x):
            if isinstance(x, list) and len(x) > 0:
                mean = np.mean(x)
                return [np.round(v - mean,3) for v in x]
            return x  # NaN이나 비리스트는 그대로 유지
        df[col] = df[col].apply(center)
    return df

def sleeptime_cutter(data): # 잠자는 시간 데이터가 더 중요한지 실험(🔥🔥🔥)

    data_filtered = data.copy()
    data_filtered['timestamp'] = pd.to_datetime(data_filtered['timestamp'])
    data_filtered['lifelog_date'] = pd.to_datetime(data_filtered['lifelog_date'])

    # spleeptime만 추출 (00시부터 06시까지)
    data_filtered = data_filtered[(data_filtered['timestamp'].dt.hour >= 0) & (data_filtered['timestamp'].dt.hour < 6)]

    # 하루 차감
    data_filtered['timestamp'] = data_filtered['timestamp'] - pd.Timedelta(days=1)
    data_filtered['lifelog_date'] = data_filtered['lifelog_date'] - pd.Timedelta(days=1)
    # print('>> D-1 하루 차감! (lifelog_date 실제 일자는 D+1 새벽(0~6시) 데이터임)')

    # lifelog_date를 다시 문자열로
    data_filtered['lifelog_date'] = data_filtered['lifelog_date'].dt.date.astype(str)

    return data_filtered

def merge_data_for_group(user, date):

    # 데이터 로드
    # acc_group = mGps.copy()
    activity_group = mActivity.copy()
    hr_group = wHr.copy()
    wPedo_group = wPedo[['subject_id','timestamp','lifelog_date','step']].copy()
    mLight_group = mLight[['subject_id','timestamp','lifelog_date','m_light']].copy()
    wLight_group = wLight[['subject_id','timestamp','lifelog_date','w_light']].copy()

    # 건수가 없는 일자 이상치로 판단하고 제외
    activity_group = filter_by_group_size(activity_group)
    hr_group = filter_by_group_size(hr_group)
    wPedo_group = filter_by_group_size(wPedo_group)
    mLight_group = filter_by_group_size(mLight_group)
    wLight_group = filter_by_group_size(wLight_group)

    # sleeptime만 남기고 나머지 삭제 (🔥🔥🔥)
    # activity_group = sleeptime_cutter(activity_group)
    # hr_group = sleeptime_cutter(hr_group)
    # wPedo_group = sleeptime_cutter(wPedo_group)
    # mLight_group = sleeptime_cutter(mLight_group)
    # wLight_group = sleeptime_cutter(wLight_group)

    # 필터
    activity_group = activity_group.loc[(activity_group['subject_id']==user) & (activity_group['lifelog_date']==date),:]
    hr_group = hr_group.loc[(hr_group['subject_id']==user) & (hr_group['lifelog_date']==date),:]
    wPedo_group = wPedo_group.loc[(wPedo_group['subject_id']==user) & (wPedo_group['lifelog_date']==date),:]
    mLight_group = mLight_group.loc[(mLight_group['subject_id']==user) & (mLight_group['lifelog_date']==date),:]
    wLight_group = wLight_group.loc[(wLight_group['subject_id']==user) & (wLight_group['lifelog_date']==date),:]

    # 리스트 평균값으로 변환
    # acc_group = average_list_columns(acc_group, ['altitude', 'latitude', 'longitude','speed'])
    hr_group = average_list_columns(hr_group, ['heart_rate'])

    # 'timestamp'를 고유하게 만듦
    # acc_group = make_timestamps_unique(acc_group)
    activity_group = make_timestamps_unique(activity_group)
    hr_group = make_timestamps_unique(hr_group)
    wPedo_group = make_timestamps_unique(wPedo_group)
    mLight_group = make_timestamps_unique(mLight_group)
    wLight_group = make_timestamps_unique(wLight_group)

    # 'timestamp'를 인덱스로 설정하고 'subject_id'와 'date' 컬럼 제거
    # mAcc_data = acc_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()
    activity_data = activity_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()
    e4Hr_data = hr_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()
    wPedo_data = wPedo_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()
    mLight_data = mLight_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()
    wLight_data = wLight_group.set_index('timestamp').drop(columns=['subject_id', 'lifelog_date']).resample('S').nearest()

    # 하루 86400초의 타임스탬프 생성
    start_time = datetime.strptime(date, '%Y-%m-%d')
    end_time = start_time + timedelta(days=1)
    all_timestamps = pd.date_range(start=start_time, end=end_time, freq='S', inclusive='left')
    merged_data = pd.DataFrame(index=all_timestamps)
    merged_data.index.name = 'timestamp'

    # 데이터 병합
    # if not mAcc_data.empty:
    #     merged_data = merged_data.join(mAcc_data, how='left')
    if not e4Hr_data.empty:
        merged_data = merged_data.join(e4Hr_data, how='left')
    if not activity_data.empty:
        merged_data = merged_data.join(activity_data, how='left')
    if not wPedo_data.empty:
        merged_data = merged_data.join(wPedo_data, how='left')
    if not mLight_data.empty:
        merged_data = merged_data.join(mLight_data, how='left')
    if not wLight_data.empty:
        merged_data = merged_data.join(wLight_data, how='left')

    # 필요한 컬럼만 유지하고 NaN 값으로 채우기
    # merged_data = merged_data.reindex(columns=['altitude', 'latitude', 'longitude', 'speed', 'heart_rate', 'm_activity', 'step'])
    merged_data = merged_data.reindex(columns=['heart_rate', 'm_activity', 'step', 'm_light', 'w_light'])

    # 선형 보간 적용
    merged_data = merged_data.interpolate(method='time')

    ### Activity 데이터의 그룹화 적용
    # group0 : 0 (IN_VEHICLE), 1 (ON_BICYCLE), 2 (ON_FOOT), 7 (WALKING), 8 (RUNNING), 5 (TILTING)
    # group1 : 3 (STILL)
    # group2 : 4 (UNKNOWN)
    activity_mapping = {
        0: 1,
        1: 1,
        2: 1,
        7: 1,
        8: 2,
        5: 1,
        3: 0,
        4: 0
    }
    merged_data['m_activity'] = merged_data['m_activity'].map(activity_mapping)

    # subject_id와 date를 추가
    merged_data['subject_id'] = user
    merged_data['lifelog_date'] = date

    return merged_data

def plot_time_series(data, user, date, channel_name):

    # x축을 00:00:00부터 23:59:59까지 고정
    total_seconds = 86400
    time_range = pd.date_range(start=datetime.strptime(date, '%Y-%m-%d'), periods=total_seconds, freq='S')

    # 데이터를 시간 단위로 정렬
    data = data.reindex(time_range)

    # 시계열 이미지 생성
    fig, axes = plt.subplots(5, 1, figsize=(5, 5), sharex=True, facecolor='black')
    fig.patch.set_facecolor('black')

    for ax in axes:
        ax.set_facecolor('black')
        ax.spines['top'].set_visible(False)           # Hide the top spine
        ax.spines['right'].set_visible(False)         # Hide the right spine
        ax.spines['left'].set_visible(False)          # Hide the left spine
        ax.spines['bottom'].set_visible(False)        # Hide the bottom spine

    # 설정한 시간 범위에 맞게 x축 설정
    for ax in axes:
        ax.set_xlim([time_range[0], time_range[-1]])

    # plot
    if 'heart_rate' in data.columns:
        axes[0].plot(data.index, data['heart_rate'], color='white')
    if 'm_activity' in data.columns:
        axes[1].plot(data.index, data['m_activity'], color='white')
    if 'step' in data.columns:
        axes[2].plot(data.index, data['step'], color='white')
    if 'm_light' in data.columns:
        axes[3].plot(data.index, data['m_light'], color='white')
    if 'w_light' in data.columns:
        axes[4].plot(data.index, data['w_light'], color='white')

    plt.tight_layout()  # Make the layout tight
    fname = f'{path}{channel_name}/user{user}_{date}_{channel_name}.png'
    plt.savefig(fname)
    # print(fname)
    # plt.show()

In [ ]:
%%time

channel_name = 'ch5_sleeptime'

# train test 데이터 합치기
a1 = train[['subject_id', 'lifelog_date']].copy()
a2 = test[['subject_id', 'lifelog_date']].copy()
val_df = pd.concat([a1,a2]).reset_index(drop=True)
print('# train:',len(train))
print('# test:',len(test))
print('# 전체 데이터:',len(val_df))

# 파일명
val_df = val_df[['subject_id', 'lifelog_date']].copy()
val_df['filename'] = val_df.apply(lambda x: f"user{x['subject_id']}_{x['lifelog_date']}_{channel_name}.png", axis=1)

# 만들어진 이미지
image_dir = f'{path}{channel_name}'
image_files = [f for f in os.listdir(image_dir) if f.endswith(f'_{channel_name}.png')]

# 남은 샘플
val_df = val_df.loc[~val_df['filename'].isin(image_files),:].reset_index(drop=True)
print('# 남은 샘플수:',len(val_df))

# ====================================
# 샘플 테스트
# ====================================
# rules = (
#   (val_df['subject_id']=='id01') & (val_df['lifelog_date'].isin(['2024-07-01']))
# )
# val_df = val_df.loc[rules,:].copy().head(1)

# 이미지 생성
bar = tqdm(range(val_df.shape[0]))
for idx in bar:
    user, date, *rest = val_df.iloc[idx].values
    bar.set_description(f'user: {user}, date: {date}')
    merged_data = merge_data_for_group(user, date)
    plot_time_series(merged_data, user, date, channel_name)

### 📌 모델 학습

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# 정사각형 패딩 후, 299x299로 리사이즈 (Xception 기준)
def preprocess_image(img):
    # 정사각형 패딩
    w, h = img.size
    max_dim = max(w, h)
    padded_img = Image.new("RGB", (max_dim, max_dim), (127, 127, 127))  # 중간값 127로 채움
    padded_img.paste(img, ((max_dim - w) // 2, (max_dim - h) // 2))

    # 리사이즈 및 [-1, 1] 정규화
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # [0,1] -> [-1,1]
    ])
    return transform(padded_img)

In [ ]:
channel_name = 'ch5_sleeptime'

# 이미지 경로 설정
dataset_path = f'{path}{channel_name}'

# 이미지 크기 설정 (Resize에 사용할 값)
image_size = 500

def find_img_mean_std(dataset_path,image_size):

  import torch
  import os
  from torchvision import transforms
  from PIL import Image

  # 전처리 파이프라인 (Normalize 제외)
  transform = transforms.Compose([
      transforms.Resize(image_size),
      transforms.ToTensor(),  # [0, 255] -> [0, 1]로 스케일링
  ])

  # PNG 파일 목록 가져오기
  image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.png')]

  # 이미지 로드 및 텐서 변환
  images = []
  for img_file in image_files:
      try:
          img = Image.open(img_file).convert('RGB')  # RGB로 강제 변환
          tensor_img = transform(img)  # [C, H, W] 형태
          images.append(tensor_img)
      except Exception as e:
          print(f"이미지 로드 실패: {img_file} - {e}")

  # 모든 이미지를 하나의 텐서로 결합
  # shape: [N, C, H, W] (N: 이미지 수, C: 채널, H: 높이, W: 너비)
  all_images = torch.stack(images, dim=0)

  # 채널별 평균 및 표준편차 계산
  # 평균: [N, C, H, W] → [C,] (모든 이미지, 모든 픽셀에 대한 평균)
  # 표준편차: 동일한 방식으로 계산
  mean = all_images.mean(dim=[0, 2, 3])  # [C,] (예: [R, G, B])
  std = all_images.std(dim=[0, 2, 3])    # [C,] (예: [R, G, B])

  # 결과 출력
  print("평균(mean):", mean.tolist())
  print("표준편차(std):", std.tolist())

  return mean.tolist(), std.tolist()

img_mean, img_std = find_img_mean_std(dataset_path,image_size)

평균(mean): [0.012695113196969032, 0.012695113196969032, 0.012695113196969032]
표준편차(std): [0.10443976521492004, 0.10443976521492004, 0.10443976521492004]


In [ ]:
def extract_cnn_features(
    image_root_dir,
    img_mean, img_std,
    nfeatures,
    batch_size=32,
    image_size=(500, 500),
    model_name='resnet50'
):
    # 이미지 확장자 허용 목록
    valid_exts = {'.png'}

    # 이미지 경로 수집
    def collect_image_paths(root_dir):
        image_paths = []
        for root, _, files in os.walk(root_dir):
            for fname in files:
                if os.path.splitext(fname)[1].lower() in valid_exts:
                    image_paths.append(os.path.join(root, fname))
        return image_paths

    # Dataset 정의
    class ImageDataset(Dataset):
        def __init__(self, image_paths, transform=None):
            self.image_paths = image_paths
            self.transform = transform

        def __len__(self):
            return len(self.image_paths)

        def __getitem__(self, idx):
            path = self.image_paths[idx]
            image = Image.open(path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, os.path.relpath(path)

    # Transform & 모델
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=img_mean, std=img_std)
    ])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 모델 선택
    # model = getattr(models, model_name)(pretrained=True)
    # feature_extractor = torch.nn.Sequential(*list(model.children())[:-1]).to(device)
    # feature_extractor.eval()

    # # 모델 설정 부분 수정
    # model = getattr(models, model_name)(pretrained=True)

    # # ResNet50의 경우, 마지막 FC 레이어 제거 후 Linear(2048 → 10) 추가
    # features = list(model.children())[:-1]  # 마지막 FC 레이어 제거
    # features.extend([
    #     torch.nn.Flatten(),  # [B, 2048, 1, 1] → [B, 2048]
    #     torch.nn.Linear(2048, nfeatures)  # 2048 → 10 차원
    # ])

    # Xception 모델 불러오기
    model = timm.create_model(model_name, pretrained=True)

    # 모든 레이어 fine-tuning 가능하게 설정
    for param in model.parameters():
        param.requires_grad = True

    # feature extractor 만들기 (특징 추출 전단만 사용 시)
    features = list(model.children())[:-1]  # 마지막 분류기 제거
    features.extend([
        torch.nn.Flatten(),
        torch.nn.Linear(model.num_features, nfeatures)  # model.num_features → 2048
    ])

    feature_extractor = torch.nn.Sequential(*features).to(device)
    feature_extractor.eval()

    # 데이터로더 생성
    image_paths = collect_image_paths(image_root_dir)
    dataset = ImageDataset(image_paths, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Feature 추출
    all_features = []
    all_names = []
    with torch.no_grad():
        for imgs, names in tqdm(dataloader):
            imgs = imgs.to(device)
            feats = feature_extractor(imgs)
            feats = feats.view(feats.size(0), -1).cpu()
            all_features.append(feats)
            all_names.extend(names)

    features_tensor = torch.cat(all_features, dim=0)
    df = pd.DataFrame(features_tensor.numpy())
    df.insert(0, 'image_path', all_names)
    fname = f"{path}img_features_{channel_name}.csv"
    df.to_csv(fname, index=False)
    print(f">> Features saved to: {fname}")

    return df

In [ ]:
# backbone_name='resnext101_32x32d'
# backbone_name='resnet18'
# resnet34	resnet18보다 깊지만 속도 손해 적음
# resnet50	널리 검증된 모델, 일반적인 이미지 작업에 매우 안정적
# efficientnet_b0	파라미터 수 적고 정확도 높음, 모바일/경량 환경에 적합
# convnext_tiny	최신 ConvNet 구조로 성능 높고 연산 효율적
# resnet101	더 깊은 네트워크로 복잡한 패턴에 강함
# efficientnet_b3 ~ b5	성능은 뛰어나지만 학습 시간이 더 큼
# convnext_base	ConvNet 중 최근 가장 높은 성능
# beit_base_patch16_224	Vision Transformer 기반, 사전학습 필수

"""
실험1 - 파라미터
[상승] nfeatures = 10
[하락] nfeatures = 5
[하락] nfeatures = 20

실험2 - 모델
[상승] resnet50
[하락] efficientnet_b0
"""

# 이미지 파생변수 생성
img_features = extract_cnn_features(
    image_root_dir=f'{path}{channel_name}',
    img_mean=img_mean, img_std=img_std,
    nfeatures = 10,
    batch_size = 32,
    image_size = (500, 500),
    model_name = 'xception'  # 다른 모델: xception, resnet50, resnet18, resnet101, efficientnet_b0
)

# check
print('# img_features.shape:',img_features.shape)
img_features.head()

100%|██████████| 22/22 [07:20<00:00, 20.02s/it]

>> Features saved to: /content/drive/MyDrive/data/ch2025_data_items/img_features_ch5_sleeptime.csv
# img_features.shape: (700, 11)


,image_path,0,1,2,3,4,5,6,7,8,9
0,drive/MyDrive/data/ch2025_data_items/ch5_sleeptime/userid01_2024-06-26_ch5_sleeptime.png,0.0036,0.0476,-0.1327,0.1057,0.1635,-0.0514,-0.0260,-0.0408,-0.2636,-0.0121
1,drive/MyDrive/data/ch2025_data_items/ch5_sleeptime/userid01_2024-06-27_ch5_sleeptime.png,0.0211,0.0250,-0.0910,0.0973,0.1678,-0.0574,0.0251,-0.0858,-0.2881,-0.0052
2,drive/MyDrive/data/ch2025_data_items/ch5_sleeptime/userid01_2024-06-28_ch5_sleeptime.png,-0.0301,-0.0700,-0.0676,0.1313,-0.0237,-0.1176,-0.0081,-0.1813,-0.3135,-0.0519
3,drive/MyDrive/data/ch2025_data_items/ch5_sleeptime/userid01_2024-06-29_ch5_sleeptime.png,0.0007,-0.0941,-0.0162,0.1509,0.0013,-0.1168,0.0285,-0.2095,-0.2874,-0.0769
4,drive/MyDrive/data/ch2025_data_items/ch5_sleeptime/userid01_2024-06-30_ch5_sleeptime.png,-0.0044,-0.0070,-0.1055,0.1611,0.0369,-0.0576,0.0404,-0.0877,-0.2168,-0.1385


In [ ]:
train = pd.read_parquet("/content/drive/MyDrive/data/train_0512.parquet")
test = pd.read_parquet("/content/drive/MyDrive/data/test_0512.parquet")

# drop_features = ['afterwork_max_label','sleeptime_max_label','worktime_max_label']
drop_features = ['top_bssid'] # ,'week_type','week_type_lag1'
drop_features = [i for i in drop_features if i in train.columns.tolist()]
print('# drop_features:',drop_features)
train = train.drop(columns=drop_features)
test = test.drop(columns=drop_features)

# drop_features: ['top_bssid']


In [ ]:
# 데이터 읽기
img_features = pd.read_csv('/content/drive/MyDrive/data/ch2025_data_items/img_features_ch5_sleeptime.csv')
img_features.columns = ['image_path']+['img'+i for i in img_features.columns if i not in ['image_path']]

# 정규표현식으로 추출
img_features['subject_id'] = img_features['image_path'].str.extract(r'user(id\d+)_')[0]
img_features['lifelog_date'] = img_features['image_path'].str.extract(r'_(\d{4}-\d{2}-\d{2})_')[0]

# check
img_features = img_features.drop(columns=['image_path'])
print(len(img_features))
display(img_features.head(1))

# add img features
train['lifelog_date'] = train['lifelog_date'].astype(str)
test['lifelog_date'] = test['lifelog_date'].astype(str)
train = train.merge(img_features,on=['subject_id','lifelog_date'],how='left')
test = test.merge(img_features,on=['subject_id','lifelog_date'],how='left')

700


,img0,img1,img2,img3,img4,img5,img6,img7,img8,img9,subject_id,lifelog_date
0,0.0036,0.0476,-0.1327,0.1057,0.1635,-0.0514,-0.0260,-0.0408,-0.2636,-0.0121,id01,2024-06-26


In [ ]:
# 요일 컬럼 추가 (예: 월요일, 화요일, ...)
train['lifelog_date'] = pd.to_datetime(train['lifelog_date'])
test['lifelog_date'] = pd.to_datetime(test['lifelog_date'])

# 요일
weekday_map = {
    0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',
    4: '금요일', 5: '토요일', 6: '일요일'
}
train['weekday'] = train['lifelog_date'].dt.dayofweek.map(weekday_map)
test['weekday'] = test['lifelog_date'].dt.dayofweek.map(weekday_map)

# 월
train['month'] = train['lifelog_date'].dt.month
test['month'] = test['lifelog_date'].dt.month

# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','일요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','일요일']),1,0)

# 공휴일
공휴일 = [
     '2024-08-15'
    ,'2024-09-16'
    ,'2024-09-17'
    ,'2024-09-18'
    ,'2024-10-03'
    ,'2024-10-09'
]
train['공휴일'] = np.where(train['lifelog_date'].isin(공휴일),1,0)
test['공휴일'] = np.where(test['lifelog_date'].isin(공휴일),1,0)

# 주말 + 공휴일 묶어주기
# train['weekend'] = np.where( ((train['weekend']==0) & (train['공휴일']==1)), 1, train['weekend'])
# test['weekend'] = np.where( ((test['weekend']==0) & (test['공휴일']==1)), 1, test['weekend'])

In [ ]:
# 추정휴가
def rule_based_sum(x):
    rules = (
        # (x['sleep_duration_min'] > (x['avg_sleep_duration'] + 30))
          (x['sleep_duration_min'] > (x['avg_sleep_duration']))
        & (x['week_type'] == 'weekday')
        & (x['month'].isin([7,8]))
    )
    return rules

train['vacation'] = train.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)
test['vacation'] = test.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)

# check
test.groupby(['subject_id'])['vacation'].sum()

,vacation
subject_id,
id01,5
id02,1
id03,5
id04,0
id05,0
id06,11
id07,14
id08,3
id09,7


In [ ]:
# 숫자형 컬럼만 선택해서 결측값 -1로 채우기
train[train.select_dtypes(include='number').columns] = train.select_dtypes(include='number').fillna(-1)
test[test.select_dtypes(include='number').columns] = test.select_dtypes(include='number').fillna(-1)

In [ ]:
def get_oof_predictions(X, y, params, n_splits=5, is_multiclass=False, num_class=None, early_stop=False):

    oof_preds = np.zeros(len(X))  # ✅ 1차원으로 변경
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        if is_multiclass:
            model = LGBMClassifier(**params, objective='multiclass', num_class=num_class)
        else:
            model = LGBMClassifier(**params)

        if early_stop:
            model.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                callbacks=[early_stopping(stopping_rounds=100, verbose=False)]
            )
        else:
            model.fit(X_train, y_train)

        preds = model.predict(X_valid)  # ✅ returns 1D array
        oof_preds[valid_idx] = preds  # ✅ 1D -> 1D 저장

    return oof_preds

In [ ]:
def run_basemodel(train, test, valid_ids, common_params, n_splits, random_state=42, early_stop=False):

    train_df = train.copy()
    test_df = test.copy()

    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    # 노이즈 수준 설정
    def add_noise(series, noise_level, seed=3):
        rng = np.random.default_rng(seed)
        return series * (1 + noise_level * rng.standard_normal(len(series)))

    noise_level = 0.015  # 필요에 따라 조정

    # 타겟인코딩
    # m = 0: 스무딩 없이 범주별 평균만 사용합니다. 관측 수가 많은 범주에는 적합하지만, 적은 경우 과적합 위험이 있습니다.
    # m = 1~10: 일반적인 기본값으로, 대부분의 상황에서 안정적인 성능을 보입니다.
    # m = 50~300: 관측 수가 매우 적은 범주가 많거나 데이터가 희소한 경우에 유용합니다.
    for tgt in all_targets:

      encoder_feats = ['subject_id','month','weekend'] # 'weekday', 'subject_id','month','weekend'

      #### 타겟인코딩1

      subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_te')
      train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
      test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
      global_mean = train_df[tgt].mean()
      test_df[f'{tgt}_te'] = test_df[f'{tgt}_te'].fillna(global_mean)

      # 노이즈 추가
      train_df[f'{tgt}_te'] = add_noise(train_df[f'{tgt}_te'], noise_level)
      test_df[f'{tgt}_te'] = add_noise(test_df[f'{tgt}_te'], noise_level)

      #### 타겟인코딩2

      # 새로운 범주형 열 생성
      train_df['TMP'] = train_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)
      test_df['TMP'] = test_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)

      # 인코더
      encoder = TargetEncoder(cols=['TMP'], smoothing=300) # 40
      encoder.fit(train_df[['TMP']], train_df[tgt])

      # 인코딩 결과를 새로운 열에 저장
      train_df[f'{tgt}_te2'] = encoder.transform(train_df[['TMP']])
      test_df[f'{tgt}_te2'] = encoder.transform(test_df[['TMP']])

      # 노이즈 추가
      train_df[f'{tgt}_te2'] = add_noise(train_df[f'{tgt}_te2'], noise_level)
      test_df[f'{tgt}_te2'] = add_noise(test_df[f'{tgt}_te2'], noise_level)

      # 불필요한 변수 제거
      train_df = train_df.drop(columns=['TMP'])
      test_df = test_df.drop(columns=['TMP'])


    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK+['pk']]
    for col in categorical_features:
        print(col)
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])


    # X
    X = train_df.drop(columns=PK + all_targets)
    test_X = test_df.drop(columns=PK + all_targets)
    print(f'# X shape: {X.shape}')
    print(f'# test_X shape: {test_X.shape}')

    print('\n STEP1: 실험 결과 확인')
    print("=============== Validation Results ==============")
    total_avg_f1s = []
    best_iteration_temp = {k: [] for k in all_targets}

    val_f1 = []
    for col in targets_binary:

        # binary
        y = train_df[col]

        valid_ids['pk'] = valid_ids['subject_id']+valid_ids['sleep_date']
        train_df['pk'] = train_df['subject_id']+train_df['sleep_date']

        X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

        best_param = best_param_dict[col].copy()
        best_param['random_state'] = random_state
        model = LGBMClassifier(**best_param)

        if early_stop:
            model.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                callbacks=[early_stopping(stopping_rounds=100,verbose=False)]
            )
            best_iteration_temp[col].append(model.best_iteration_)
        else:
            model.fit(X_train, y_train)
            best_iteration_temp[col].append(1000)

        pred_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, pred_valid, average='macro') ### 수정
        val_f1.append(f1)

    # multi
    y = train_df[target_multiclass]

    X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    best_param = best_param_dict['S1'].copy()
    best_param['random_state'] = random_state
    model = LGBMClassifier(**best_param, objective='multiclass', num_class=3)

    if early_stop:
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            callbacks=[early_stopping(stopping_rounds=100,verbose=False)]
        )
        best_iteration_temp[target_multiclass].append(model.best_iteration_)
    else:
        model.fit(X_train, y_train)
        best_iteration_temp[target_multiclass].append(1000)

    pred_valid = model.predict(X_valid)
    f1 = f1_score(y_valid, pred_valid, average='macro')
    val_f1.append(f1)


    avg_f1 = np.mean(val_f1)
    total_avg_f1s.append(avg_f1)
    detail = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['S1'], val_f1)])
    print(f" 평균 F1: {avg_f1:.4f} / [상세] {detail}")

    best_iteration_dict = {k: max(best_iteration_temp[k]) for k in all_targets}

    if early_stop==True:
      print("\n[best_iteration_dict]")
      for k, v in best_iteration_dict.items():
          print(f"{k}: {v}")


    print(f"# 전체 평균 F1: {np.mean(total_avg_f1s):.4f}")
    print("================================================")

    # modoling with 100% train & no valid
    print('\n STEP2: 전체 데이터로 모델 재학습')
    print("====== modoling with 100% train & no valid =====")

    # binary
    binary_preds = {}
    for col in targets_binary:
        binary_params = best_param_dict[col].copy()
        binary_params['random_state'] = random_state
        y = train_df[col]

        if early_stop:
          binary_params['n_estimators']=best_iteration_dict[col]
          model = LGBMClassifier(**binary_params)
          model.fit(X, y)
        else:
          model = LGBMClassifier(**binary_params)
          model.fit(X, y)

        binary_preds[col] = model.predict(test_X)
        fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
        top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
        feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
        print(f"[{col}] {feat_str}")

    # multiclass
    y = train_df['S1']
    binary_params = best_param_dict['S1'].copy()
    binary_params['random_state'] = random_state

    if early_stop:
      binary_params['n_estimators']=best_iteration_dict['S1']
      model = LGBMClassifier(**binary_params)
      model.fit(X, y)
    else:
      model = LGBMClassifier(**binary_params)
      model.fit(X, y)

    multiclass_pred = model.predict(test_X)
    fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
    feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
    print(f"[S1] {feat_str}")

    # 예측 저장
    submission_final['S1'] = multiclass_pred
    for col in targets_binary:
      submission_final[col] = binary_preds[col]
    submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
    fname = f"/content/drive/MyDrive/data/submission_{np.mean(total_avg_f1s)}.csv"
    submission_final.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")
    print(f"# submission shape:{submission_final.shape}")
    print("================================================")

    # 모델별 예측결과 비율 비교
    a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    a21 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a23 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a22 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
    result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
    print('\n STEP3: 예측결과 비교표')
    display(result)

    # === STEP4: OOF 예측 생성 (train set에 대해) ===

    # n_splits = 10
    mask = train['month'] != 6
    print(f'# k-fold: {n_splits}')
    print(f'# train: {len(y[mask])}')

    oof_f1 = []
    print('\n STEP4: OOF 예측 생성')
    oof_result = train_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    for col in targets_binary:
        params = best_param_dict[col].copy()
        params['random_state'] = random_state
        y = train_df[col]
        oof_preds = get_oof_predictions(X, y, params, n_splits=n_splits, is_multiclass=False, early_stop=early_stop)
        oof_result[col] = oof_preds
        f1 = f1_score(y[mask], oof_preds[mask], average='macro')
        oof_f1.append(f1)
        print(f"[OOF - {col}] F1 score: {f1:.4f}")

    # multiclass
    col = 'S1'
    params = best_param_dict[col].copy()
    params['random_state'] = random_state
    y = train_df[col]
    oof_preds = get_oof_predictions(X, y, params, n_splits=n_splits, is_multiclass=True, num_class=3, early_stop=early_stop)
    oof_result[col] = oof_preds
    f1 = f1_score(y[mask], oof_preds[mask], average='macro')
    oof_f1.append(f1)
    print(f"[OOF - {col}] F1 score: {f1:.4f}")
    print(f"[OOF] F1 score: {np.mean(oof_f1):.4f}")

    # oof_result 저장
    fname = f"/content/drive/MyDrive/data/oof_result_{np.mean(total_avg_f1s)}.csv"
    oof_result.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")

    return submission_final, oof_result

### run

In [ ]:
"""
week_type
week_type_lag1
weekday
# X shape: (450, 168)
# test_X shape: (250, 168)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
 평균 F1: 0.6322 / [상세] Q1(기상직후수면질):0.7278 Q2(취침전신체적피로):0.7122 Q3(취침전스트레스):0.6830 S2(수면효율):0.5726 S3(수면잠들기시간):0.6686 S1(S1):0.4293
# 전체 평균 F1: 0.6322
================================================

 STEP2: 전체 데이터로 모델 재학습
====== modoling with 100% train & no valid =====
[Q1] Q1_te2(557), light_night_mean(469), wake_time_ratio(405), wake_time_diff_lag1(340), 통화_time(326), Q1_te(325), sleep_duration_diff(242), activehour_unique_label_count(200), ble_class_others_ratio_worktime(175), wake_time(155)
[Q2] Q2_te2(2791), total_screen_time(351), wake_up_early_minutes(331), speed_le5_max(307), rolling_wake_time_3d(288), rolling_sleep_time_3d(233), sleep_time_diff(224), Q2_te(198), wlight_evening_mean(182), hr_evening_std(176)
[Q3] Q3_te2(2199), light_max(366), sleep_duration_diff_lag1(274), sleep_duration_min(227), sleep_duration_ratio(211), screen_time_vs_avg_pct(187), lat_change(181), all_VEHICLE_minutes(179), sleep_time(177), 통화_time(167)
[S2] S2_te(3725), S2_te2(2773), wake_time_diff_lag1(340), light_max(292), light_night_mean(245), S1_te2(189), ble_class_unknwn_ratio_sleeptime(179), 통화_time(175), sleep_duration_lag1(171), sleep_duration_min(170)
[S3] S3_te(639), light_night_mean(443), S3_te2(336), ble_rssi_mean_afterwork(256), hr_evening_min(242), activehour_unique_label_count(242), ble_class_unknwn_ratio_sleeptime(235), wlight_evening_mean(231), sleep_time_diff_lag1(230), vehicle_minutes(180)
[S1] S1_te2(3990), S1_te(661), sleep_duration_ratio(646), wake_time_ratio(554), sleep_duration_diff(452), sleep_duration_min(429), vehicle_minutes(427), rolling_wake_time_3d(410), speed_le5_max(408), hour_span_minutes(379)
# /content/drive/MyDrive/data/submission_0.6322252622334963.csv 저장 완료
# submission shape:(250, 9)
================================================

 STEP3: 예측결과 비교표
학습sum	학습len	학습mean	테스트sum	테스트len	테스트mean
Q1	223	450	0.4956	131	250	0.5240
Q2	253	450	0.5622	150	250	0.6000
Q3	270	450	0.6000	173	250	0.6920
S1	390	450	0.8667	202	250	0.8080
S2	293	450	0.6511	170	250	0.6800
S3	298	450	0.6622	171	250	0.6840


# k-fold: 5
# train: 392

 STEP4: OOF 예측 생성
[OOF - Q1] F1 score: 0.6976
[OOF - Q2] F1 score: 0.7041
[OOF - Q3] F1 score: 0.6605
[OOF - S2] F1 score: 0.6610
[OOF - S3] F1 score: 0.7106
[OOF - S1] F1 score: 0.5233
[OOF] F1 score: 0.6595
# /content/drive/MyDrive/data/oof_result_0.6322252622334963.csv 저장 완료
"""

# 공통 하이퍼파라미터
common_params = {
  'n_estimators': 5000,
  "learning_rate": 0.01,
  # "shrinkage_rate": 0.12,
  # 'min_data_in_leaf':2,
  # 'bagging_fraction':0.9,
  # 'feature_fraction':0.6,
  'lambda_l1': 5,
  'lambda_l2': 1,
  # 'max_depth': 4,
  'n_jobs': -1,
  'verbosity': -1
}

# 모델별 세부 하이퍼파라미터
best_param_dict = {'Q1': {'learning_rate': 0.1473150575266255,
  'shrinkage_rate': 0.08585454450680065,
  'min_data_in_leaf': 13,
  'bagging_fraction': 0.5900885111562433,
  'feature_fraction': 0.7398526832500182,
  'lambda_l1': 0.7309384079752819,
  'lambda_l2': 0.010419978985191203,
  'max_depth': 2},
 'Q2': {'learning_rate': 0.1433742819325529,
  'shrinkage_rate': 0.4777741359643458,
  'min_data_in_leaf': 11,
  'bagging_fraction': 0.8942012129234453,
  'feature_fraction': 0.3442323511952453,
  'lambda_l1': 0.11108296857244106,
  'lambda_l2': 0.5000682520529595,
  'max_depth': 11},
 'Q3': {'learning_rate': 0.005440413154494791,
  'shrinkage_rate': 0.4869550654391126,
  'min_data_in_leaf': 5,
  'bagging_fraction': 0.992720410336095,
  'feature_fraction': 0.10854085794750301,
  'lambda_l1': 8.765258863766789,
  'lambda_l2': 0.010911793484805324,
  'max_depth': -1},
 'S1': {'learning_rate': 0.19808502263166988,
  'shrinkage_rate': 0.3292477285579064,
  'min_data_in_leaf': 9,
  'bagging_fraction': 0.5929013243246726,
  'feature_fraction': 0.8481981135327139,
  'lambda_l1': 0.010377995886618164,
  'lambda_l2': 0.6226891522266145,
  'max_depth': 10},
 'S2': {'learning_rate': 0.27099064035077214,
  'shrinkage_rate': 0.028901883938906636,
  'min_data_in_leaf': 9,
  'bagging_fraction': 0.8134249396247819,
  'feature_fraction': 0.2321570003912355,
  'lambda_l1': 8.780092357464005,
  'lambda_l2': 9.605716023562762,
  'max_depth': 7},
 'S3': {'learning_rate': 0.14542046442644,
  'shrinkage_rate': 0.3047247759570036,
  'min_data_in_leaf': 10,
  'bagging_fraction': 0.8493532899163512,
  'feature_fraction': 0.7940889257506005,
  'lambda_l1': 9.299803284110112,
  'lambda_l2': 0.12938944891518922,
  'max_depth': 6}
}

# 공통 하이퍼파라미터 대체 (이상한 모델의 경우)
best_param_dict['Q3'] = common_params
best_param_dict['S1'] = common_params
best_param_dict['S2'] = common_params
best_param_dict['S3'] = common_params
best_param_dict['Q1'] = common_params
best_param_dict['Q2'] = common_params

# 전체 평균 F1: 0.6322
# [OOF] F1 score: 0.6595
# [OOF - Q1] F1 score: 0.6976
# [OOF - Q2] F1 score: 0.7041
# [OOF - Q3] F1 score: 0.6605
# [OOF - S2] F1 score: 0.6610
# [OOF - S3] F1 score: 0.7106
# [OOF - S1] F1 score: 0.5233
submission_final, oof_result = run_basemodel(train, test, valid_ids, best_param_dict, n_splits=5, random_state=41, early_stop=False)

week_type
week_type_lag1
weekday
# X shape: (450, 190)
# test_X shape: (250, 190)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
 평균 F1: 0.6448 / [상세] Q1(기상직후수면질):0.7314 Q2(취침전신체적피로):0.7518 Q3(취침전스트레스):0.6546 S2(수면효율):0.6179 S3(수면잠들기시간):0.6485 S1(S1):0.4642
# 전체 평균 F1: 0.6448

 STEP2: 전체 데이터로 모델 재학습
====== modoling with 100% train & no valid =====
[Q1] Q1_te2(539), light_night_mean(373), wake_time_ratio(360), wake_time_diff_lag1(290), Q1_te(262), sleep_duration_diff(257), wake_time(192), beforebed_전화_time(179), beforebed_통화_time(178), ble_rssi_max_afterwork(174)
[Q2] Q2_te2(1510), rolling_wake_time_3d(250), wake_up_early_minutes(249), wlight_evening_mean(230), Q2_te(229), activehour_total_screen_time(193), rolling_sleep_time_3d(179), activehour_screen_time_vs_avg_pct(170), avg_rssi(167), activehour_전화_time(161)
[Q3] Q3_te2(641), light_max(403), sleep_duration_min(241), activehour_통화_time(236), sleep_duration_diff_lag1(210), sleep_duration_lag1(164), ble_class_other

,학습sum,학습len,학습mean,테스트sum,테스트len,테스트mean
Q1,223,450,0.4956,127,250,0.5080
Q2,253,450,0.5622,151,250,0.6040
Q3,270,450,0.6000,174,250,0.6960
S1,390,450,0.8667,199,250,0.7960
S2,293,450,0.6511,166,250,0.6640
S3,298,450,0.6622,167,250,0.6680


# k-fold: 5
# train: 392

 STEP4: OOF 예측 생성
[OOF - Q1] F1 score: 0.6980
[OOF - Q2] F1 score: 0.6873
[OOF - Q3] F1 score: 0.6779
[OOF - S2] F1 score: 0.6678
[OOF - S3] F1 score: 0.7158
[OOF - S1] F1 score: 0.5464
[OOF] F1 score: 0.6655
# /content/drive/MyDrive/data/oof_result_0.6447528574485211.csv 저장 완료
